In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import os

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset
#school_data_complete = pd.merge(student_data, school_data, how="left", on=["name"])
#school_data_complete.head()



In [2]:
school_data.head()
#school_data.max()

,School ID,name,type,size,budget
0,0,Huang High School,District,2917,1910635
1,1,Figueroa High School,District,2949,1884411
2,2,Shelton High School,Charter,1761,1056600
3,3,Hernandez High School,District,4635,3022020
4,4,Griffin High School,Charter,1468,917500


In [3]:
student_data.head()

,Student ID,name,gender,grade,school,reading_score,math_score
0,0,Paul Bradley,M,9th,Huang High School,66,79
1,1,Victor Smith,M,12th,Huang High School,94,61
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58
4,4,Bonnie Ray,F,9th,Huang High School,97,84


In [4]:

### District Summary


#Unique schools names
school_names= school_data["name"].unique()
#Number of Schools
school_count=len(school_names)
#school_count


#Number of Students
student_count=len(student_data)
#student_count

#Total Budget
total_budget=sum(school_data["budget"])
#total_budget

#Average Math score
avg_math_score= sum(student_data["math_score"])/student_count
#avg_math_score

#Average Reading score
avg_reading_score= (student_data["reading_score"]).mean()
#avg_reading_score

# % Passing Math
math_passing = student_data.loc[student_data["math_score"]>=70]["math_score"].count()
#math_passing
math_passing_percent= (math_passing)/student_count
#math_passing_percent


# % Passing Reading
reading_passing = student_data.loc[student_data["reading_score"]>=70]["reading_score"].count()
reading_passing_percent=(reading_passing)/student_count
#reading_passing_percent

# Overall Passing Rate (Average of the above two)
overall_passing= np.mean([reading_passing_percent, math_passing_percent])
#overall_passing


#District Summary in a table
district_summary = pd.DataFrame({
    
    "Total Schools": [school_count],
    "Total Students": [student_count],
    "Total Budget": [total_budget],
    "Average Reading Score": [avg_reading_score],
    "Average Math Score": [avg_math_score],
    "% Passing Reading":[reading_passing_percent],
    "% Passing Math": [math_passing_percent],
    "Overall Passing Rate": [overall_passing]

})

#store as different df to change order
dist_sum = district_summary[["Total Schools", "Total Students", "Total Budget", "Average Reading Score", "Average Math Score", '% Passing Reading', '% Passing Math', 'Overall Passing Rate']]

#format cells
dist_sum.style.format({"Total Budget": "${:,.2f}", "Average Reading Score": "{:.1f}", "Average Math Score": "{:.1f}", "% Passing Math": "{:.1%}", "% Passing Reading": "{:.1%}", "Overall Passing Rate": "{:.1%}"})

,Total Schools,Total Students,Total Budget,Average Reading Score,Average Math Score,% Passing Reading,% Passing Math,Overall Passing Rate
0,15,39170,"$24,649,428.00",81.9,79.0,85.8%,75.0%,80.4%


In [5]:
### School Summary"school

#School Name
schoolnames = student_data.groupby(['school'])


#School Type
#Total Students
#total_students_per_school = pd.DataFrame([schoolnames['Student ID'].count(), schoolnames])                                 
#Total School Budget


#Per Student Budget
school_data['Per Student Budget'] = school_data['budget']/school_data['size']
school_data.rename(columns = {'name': 'school'}, inplace = True)


#% Passing Math
#school_student_data= student_data["school"].unique()
avg_math = schoolnames['math_score'].mean().round(1).reset_index()


#% Passing Reading
avg_read = schoolnames['reading_score'].mean().round(1).reset_index()


#Overall Passing Rate (Average of the above two)
avg_scores = pd.merge(avg_math, avg_read, on=('school'))
avg_scores.rename(columns = {'math_score': 'Average Math Score', 'reading_score': 'Average Reading Score'}, inplace=True)




#creates dataframe of avg math and reading score by school


#school level passing scores counts by using conditional and only keeping school and math score
pass_math = student_data.loc[student_data['math_score'] >=70][['school', 'math_score']]
pass_math_by_sch = pass_math.groupby('school').count().reset_index()
pass_math_by_sch.rename(columns = {"math_score": "# passing math"}, inplace=True)

#reading
pass_read = student_data.loc[student_data['reading_score'] >=70][['school', 'reading_score']]
pass_read_by_sch = pass_read.groupby('school').count().reset_index()
pass_read_by_sch.rename(columns = {"reading_score": "# passing reading"}, inplace=True)

#merge math and reading data
pass_count = pd.merge(pass_math_by_sch, pass_read_by_sch, on=('school'))


#merge all on school
sch_summary = pd.merge(school_data, avg_scores, on=('school'))
sch_summary = pd.merge(sch_summary, pass_count, on=('school'))


# adding percent passing columns
sch_summary['% Passing Math'] = sch_summary['# passing math']/sch_summary['size']
sch_summary['% Passing Reading'] = sch_summary['# passing reading']/sch_summary['size']

#delete extraneous columns
del sch_summary['# passing math']
del sch_summary['# passing reading']

# create Overall Passing Rat columns
sch_summary['Overall Passing Rate'] = (sch_summary['% Passing Math']+sch_summary['% Passing Reading'])/2
#formatting and adjustments for aesthetics
sch_summary.rename(columns = {'school': "School Name", "type": "School Type", "size":"Total Students", "budget": "Total School Budget"}, inplace = True)
sch_summary.set_index('School Name', inplace=True)
sch_summary.style.format({'Total Students': '{:,}', "Total School Budget": "${:,}", "Per Student Budget": "${:.0f}", "% Passing Math": "{:.1%}", "% Passing Reading": "{:.1%}", "Overall Passing Rate": "{:.1%}"})



,School ID,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
School Name,,,,,,,,,,
Huang High School,0,District,"2,917","$1,910,635",$655,76.6,81.2,65.7%,81.3%,73.5%
Figueroa High School,1,District,"2,949","$1,884,411",$639,76.7,81.2,66.0%,80.7%,73.4%
Shelton High School,2,Charter,"1,761","$1,056,600",$600,83.4,83.7,93.9%,95.9%,94.9%
Hernandez High School,3,District,"4,635","$3,022,020",$652,77.3,80.9,66.8%,80.9%,73.8%
Griffin High School,4,Charter,"1,468","$917,500",$625,83.4,83.8,93.4%,97.1%,95.3%
Wilson High School,5,Charter,"2,283","$1,319,574",$578,83.3,84,93.9%,96.5%,95.2%
Cabrera High School,6,Charter,"1,858","$1,081,356",$582,83.1,84,94.1%,97.0%,95.6%
Bailey High School,7,District,"4,976","$3,124,928",$628,77,81,66.7%,81.9%,74.3%
Holden High School,8,Charter,427,"$248,087",$581,83.8,83.8,92.5%,96.3%,94.4%


In [6]:
### Top performance school by passing rate
top_per_schls = sch_summary.sort_values("Overall Passing Rate", ascending = False)
top_per_schls.head().style.format({'Total Students': '{:,}', "Total School Budget": "${:,}", "Per Student Budget": "${:.0f}", "% Passing Math": "{:.1%}", "% Passing Reading": "{:.1%}", "Overall Passing Rate": "{:.1%}"})

,School ID,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
School Name,,,,,,,,,,
Cabrera High School,6,Charter,"1,858","$1,081,356",$582,83.1,84,94.1%,97.0%,95.6%
Thomas High School,14,Charter,"1,635","$1,043,130",$638,83.4,83.8,93.3%,97.3%,95.3%
Pena High School,9,Charter,962,"$585,858",$609,83.8,84,94.6%,95.9%,95.3%
Griffin High School,4,Charter,"1,468","$917,500",$625,83.4,83.8,93.4%,97.1%,95.3%
Wilson High School,5,Charter,"2,283","$1,319,574",$578,83.3,84,93.9%,96.5%,95.2%


In [7]:
### Bottom Performing Schools (By Passing Rate)

bottom_per_schls = sch_summary.sort_values("Overall Passing Rate", ascending = True)
bottom_per_schls.head().style.format({'Total Students': '{:,}', "Total School Budget": "${:,}", "Per Student Budget": "${:.0f}", "% Passing Math": "{:.1%}", "% Passing Reading": "{:.1%}", "Overall Passing Rate": "{:.1%}"})


,School ID,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
School Name,,,,,,,,,,
Rodriguez High School,11,District,"3,999","$2,547,363",$637,76.8,80.7,66.4%,80.2%,73.3%
Figueroa High School,1,District,"2,949","$1,884,411",$639,76.7,81.2,66.0%,80.7%,73.4%
Huang High School,0,District,"2,917","$1,910,635",$655,76.6,81.2,65.7%,81.3%,73.5%
Johnson High School,12,District,"4,761","$3,094,650",$650,77.1,81,66.1%,81.2%,73.6%
Ford High School,13,District,"2,739","$1,763,916",$644,77.1,80.7,68.3%,79.3%,73.8%


In [8]:
## Math Scores by Grade\*\*

# Craeting bins for grade level
ninth_m = student_data.loc[student_data['grade'] == '9th'].groupby('school')["math_score"].mean().reset_index()
ninth_m.rename(columns = {'math_score': "9th"}, inplace=True)
tenth_m = student_data.loc[student_data['grade'] == '10th'].groupby('school')["math_score"].mean().reset_index()
tenth_m.rename(columns = {'math_score': "10th"}, inplace=True)
eleventh_m = student_data.loc[student_data['grade'] == '11th'].groupby('school')["math_score"].mean().reset_index()
eleventh_m.rename(columns = {'math_score': "11th"}, inplace=True)
twelfth_m = student_data.loc[student_data['grade'] == '12th'].groupby('school')["math_score"].mean().reset_index()
twelfth_m.rename(columns = {'math_score': "12th"}, inplace=True)

#merges the math score averages by school and grade together
math_scores = pd.merge(ninth_m, tenth_m, on = 'school').merge(eleventh_m, on = 'school').merge(twelfth_m, on = 'school')

#formatting
math_scores.rename(columns = {'school':'School Name'}, inplace = True)
math_scores.set_index('School Name', inplace = True)
math_scores.style.format({'9th': '{:.2f}', "10th": '{:.2f}', "11th": "{:.2f}", "12th": "{:.2f}"})

,9th,10th,11th,12th
School Name,,,,
Bailey High School,77.08,77.00,77.52,76.49
Cabrera High School,83.09,83.15,82.77,83.28
Figueroa High School,76.40,76.54,76.88,77.15
Ford High School,77.36,77.67,76.92,76.18
Griffin High School,82.04,84.23,83.84,83.36
Hernandez High School,77.44,77.34,77.14,77.19
Holden High School,83.79,83.43,85.00,82.86
Huang High School,77.03,75.91,76.45,77.23
Johnson High School,77.19,76.69,77.49,76.86


In [9]:
### Reading Scores by Grade

# Craeting bins for grade level
ninth_r = student_data.loc[student_data['grade'] == '9th'].groupby('school')["math_score"].mean().reset_index()
ninth_r.rename(columns = {'reading_score': "9th"}, inplace=True)
tenth_r = student_data.loc[student_data['grade'] == '10th'].groupby('school')["math_score"].mean().reset_index()
tenth_r.rename(columns = {'reading_score': "10th"}, inplace=True)
eleventh_r = student_data.loc[student_data['grade'] == '11th'].groupby('school')["math_score"].mean().reset_index()
eleventh_r.rename(columns = {'reading_score': "11th"}, inplace=True)
twelfth_r = student_data.loc[student_data['grade'] == '12th'].groupby('school')["math_score"].mean().reset_index()
twelfth_r.rename(columns = {'reading_score': "12th"}, inplace=True)

#merges the math score averages by school and grade together
reading_scores = pd.merge(ninth_r, tenth_r, on = 'school').merge(eleventh_r, on = 'school').merge(twelfth_r, on = 'school')

#formatting
reading_scores.rename(columns = {'school':'School Name'}, inplace = True)
reading_scores.set_index('School Name', inplace = True)
math_scores.style.format({'9th': '{:.2f}', "10th": '{:.2f}', "11th": "{:.2f}", "12th": "{:.2f}"})


,9th,10th,11th,12th
School Name,,,,
Bailey High School,77.08,77.00,77.52,76.49
Cabrera High School,83.09,83.15,82.77,83.28
Figueroa High School,76.40,76.54,76.88,77.15
Ford High School,77.36,77.67,76.92,76.18
Griffin High School,82.04,84.23,83.84,83.36
Hernandez High School,77.44,77.34,77.14,77.19
Holden High School,83.79,83.43,85.00,82.86
Huang High School,77.03,75.91,76.45,77.23
Johnson High School,77.19,76.69,77.49,76.86


In [10]:
### Scores by School Spending


school_data.rename(columns = {'name': 'school'}, inplace = True)

student_school_merged = student_data.merge(school_data, how = 'left', on = 'school')


#add a column and assign bins for spending per student
student_school_merged.loc[(student_school_merged['Per Student Budget'] < 585), "spending_bin"] = "< $585"
student_school_merged.loc[((student_school_merged['Per Student Budget'] >= 585) & (student_school_merged['Per Student Budget'] < 615)), "spending_bin"] = "$585 - 614" 
student_school_merged.loc[((student_school_merged['Per Student Budget'] >= 615) & (student_school_merged['Per Student Budget'] < 645)), "spending_bin"] = "$615 - 644" 
student_school_merged.loc[(student_school_merged['Per Student Budget'] >= 645), "spending_bin"] = "> $644" 

# group by spending bin
by_spending = student_school_merged.groupby('spending_bin')

# average math and reading score for each spending bin
avg_scores_by_spend = by_spending['math_score', 'reading_score'].mean().reset_index()

# no. of students passing in each spending bin by using conditional and rename column
pass_read_by_spend = student_school_merged[student_school_merged['reading_score'] >= 70].groupby('spending_bin')['reading_score'].count().reset_index()
pass_read_by_spend.rename(columns = {'reading_score': '# pass reading'}, inplace=True)

pass_math_by_spend = student_school_merged[student_school_merged['math_score'] >= 70].groupby('spending_bin')['math_score'].count().reset_index()
pass_math_by_spend.rename(columns = {'math_score': '# pass math'}, inplace=True)

# no. of students in each spending bin to calculate percentage
size_by_spend = by_spending['name'].count().reset_index()
size_by_spend.rename(columns = {'name':'size'}, inplace = True)

# merge
scores_by_spend = pd.merge(avg_scores_by_spend, pass_read_by_spend, on = "spending_bin").merge(pass_math_by_spend, on='spending_bin').merge(size_by_spend, on='spending_bin')
# add columns for % passing math and reading
scores_by_spend['% Passing Reading'] = scores_by_spend['# pass reading']/scores_by_spend['size']
scores_by_spend['% Passing Math'] = scores_by_spend['# pass math']/scores_by_spend['size']
# keep only data needed for table
scores_by_spend = scores_by_spend[['spending_bin', 'math_score', 'reading_score', '% Passing Math', '% Passing Reading']]
# add column for overall passing rate
scores_by_spend['Overall Passing Rate'] = (scores_by_spend['% Passing Reading']+ scores_by_spend['% Passing Math'])/2
#reorder rows
scores_by_spend = scores_by_spend.reindex([2,0,1,3])
#formatting
scores_by_spend.rename(columns = {'spending_bin':'Spending Per Student','math_score': 'Average Math Score', 'reading_score':'Average Reading Score'}, inplace=True)
scores_by_spend.set_index('Spending Per Student', inplace=True)
scores_by_spend.style.format({'Average Math Score': '{:.1f}', 'Average Reading Score': '{:.1f}', '% Passing Math': '{:.1%}', '% Passing Reading':'{:.1%}', 'Overall Passing Rate': '{:.1%}'})


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Spending Per Student,,,,,
< $585,83.4,84.0,93.7%,96.7%,95.2%
$585 - 614,83.5,83.8,94.1%,95.9%,95.0%
$615 - 644,78.1,81.4,71.4%,83.6%,77.5%
> $644,77.0,81.0,66.2%,81.1%,73.7%


In [11]:
### Scores by school size




# bin by number of students in a school
student_school_merged.loc[(student_school_merged['size'] < 1000), "size_class"] = "Small (<1000)"
student_school_merged.loc[((student_school_merged['size'] >= 1000) & (student_school_merged['size'] <= 2000)), "size_class"] = "Medium (1000-2000)" 
student_school_merged.loc[((student_school_merged['size'] >= 2000) & (student_school_merged['size'] <= 5000)), "size_class"] = "Large (2000-5000)"  

#groupby school size bins
by_size = student_school_merged.groupby("size_class")

# no. average scores for math and reading by size bin
avg_scores_by_size = by_size['math_score', 'reading_score'].mean().reset_index()

# no. of students passing math and reading by size of school
pass_read_by_size = student_school_merged[student_school_merged['reading_score'] >= 70].groupby("size_class")['reading_score'].count().reset_index()
pass_read_by_size.rename(columns = {'reading_score': '# pass reading'}, inplace=True)

pass_math_by_size = student_school_merged[student_school_merged['math_score'] >= 70].groupby("size_class")['math_score'].count().reset_index()
pass_math_by_size.rename(columns = {'math_score': '# pass math'}, inplace=True)

#get number of students in each size bin
size_by_size = by_size['name'].count().reset_index()
size_by_size.rename(columns = {'name':'size'}, inplace = True)

#merge to use in calculation
scores_by_size = pd.merge(avg_scores_by_size, pass_read_by_size, on = "size_class").merge(pass_math_by_size, on="size_class").merge(size_by_size, on="size_class")
#calculate %s
scores_by_size['% Passing Reading'] = scores_by_size['# pass reading']/scores_by_size['size']
scores_by_size['% Passing Math'] = scores_by_size['# pass math']/scores_by_size['size']
# get rid of columns not needed
scores_by_size = scores_by_size[["size_class", 'math_score', 'reading_score', '% Passing Math', '% Passing Reading']]
# calculate overall passing rate
scores_by_size['Overall Passing Rate'] = (scores_by_size['% Passing Reading']+ scores_by_size['% Passing Math'])/2

#formatting
scores_by_size = scores_by_size.reindex([2,1,0])
scores_by_size.rename(columns = {"size_class": "School Size",'math_score': 'Average Math Score', 'reading_score':'Average Reading Score'}, inplace=True)
scores_by_size.set_index('School Size', inplace=True)
scores_by_size.style.format({'Average Math Score': '{:.1f}', 'Average Reading Score': '{:.1f}', '% Passing Math': '{:.1%}', '% Passing Reading':'{:.1%}', 'Overall Passing Rate': '{:.1%}'})

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
School Size,,,,,
Small (<1000),83.8,84.0,94.0%,96.0%,95.0%
Medium (1000-2000),83.4,83.9,93.6%,96.8%,95.2%
Large (2000-5000),77.5,81.2,68.7%,82.1%,75.4%


In [12]:
### Scores by School Type

# group by type of school
by_type = student_school_merged.groupby("type")

# average score by type
avg_scores_by_type = by_type['math_score', 'reading_score'].mean().reset_index()

# no. passing by type of school
pass_read_by_type = student_school_merged[student_school_merged['reading_score'] >= 70].groupby("type")['reading_score'].count().reset_index()
pass_read_by_type.rename(columns = {'reading_score': '# pass reading'}, inplace=True)

pass_math_by_type = student_school_merged[student_school_merged['math_score'] >= 70].groupby("type")['math_score'].count().reset_index()
pass_math_by_type.rename(columns = {'math_score': '# pass math'}, inplace=True)

# no. of students by type of school
size_by_type = by_type['name'].count().reset_index()
size_by_type.rename(columns = {'name':'size'}, inplace = True)

# merge data for calculations
scores_by_type = pd.merge(avg_scores_by_type, pass_read_by_type, on = "type").merge(pass_math_by_type, on="type").merge(size_by_type, on="type")
scores_by_type['% Passing Reading'] = scores_by_type['# pass reading']/scores_by_type['size']
scores_by_type['% Passing Math'] = scores_by_type['# pass math']/scores_by_type['size']
# only keep needed columns
scores_by_type = scores_by_type[["type", 'math_score', 'reading_score', '% Passing Math', '% Passing Reading']]
# calc passing rate for each type
scores_by_type['Overall Passing Rate'] = (scores_by_type['% Passing Reading']+ scores_by_type['% Passing Math'])/2
# formatting
scores_by_type.rename(columns = {"type": "School Size",'math_score': 'Average Math Score', 'reading_score':'Average Reading Score'}, inplace=True)
scores_by_type.set_index('School Size', inplace=True)
scores_by_type.style.format({'Average Math Score': '{:.1f}', 'Average Reading Score': '{:.1f}', '% Passing Math': '{:.1%}', '% Passing Reading':'{:.1%}', 'Overall Passing Rate': '{:.1%}'})

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
School Size,,,,,
Charter,83.4,83.9,93.7%,96.6%,95.2%
District,77.0,81.0,66.5%,80.9%,73.7%
